In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, SimpleRNN, Input, Activation, Dropout
from keras import backend as K
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow as tf
from keras.models import Model
from keras.models import load_model
import time #to calculate the computation time

2022-10-13 01:36:22.539345: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-13 01:36:22.539424: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# specifying constant parameters
#input varioable: Q, C_A0
#state variable: T, C_A

T_0 = 300
V = 1
k_0 = 8.46*(np.power(10,6))
C_p = 0.231
rho_L = 1000
F = 5
E = 5*(np.power(10,4))
delta_H = -1.15*(np.power(10,4))
R = 8.314



T_0 += 0.1 * T_0
V += 0.1 * V
k_0 += 0.1 * k_0
C_p += 0.1 * C_p
rho_L += 0.1 * rho_L
F += 0.1 * F
E += 0.1 * E
delta_H += 0.1 * delta_H
R = R    #constant
  
Q_s = 0.0  # the steady state for input variable Q
C_A0s = 4.2   # the steady state for input variable C_A0

#calculated by let dCA/dt,dT/dt =0, via scipy
T_s = 333.47  #151997  # the steady state for state variable T
C_As = 4.12  #32945  # the steady state for state variable C_A

t_final = 0.01  #the control period
t_step = 1e-4   # the step to use first-principle to calculate the state
P = np.array([[1060, 22], [22, 0.52]])

In [3]:
#The data size for new-cstr should be consensus with the basic-one
# generating inputs and initial states for CSTR, all expressed in deviation form

# Test 1 full stability region
u1_list = np.linspace(-3.5, 3.5, 30, endpoint=True)
u2_list = np.linspace(-5e5, 5e5, 30, endpoint=True)
T_initial = np.linspace(200, 550, 30, endpoint=True) - T_s
CA_initial = np.linspace(0, 8., 30, endpoint=True) - C_As
#print(T_initial)
#control variable: C_A0, Q
#state variable: C_A, T

# sieve out initial states that lie outside of stability region

T_start = list()
CA_start = list()

for T in T_initial:   
    for CA in CA_initial:
        x = np.array([CA, T])
        if x @ P @ x < 480:   #calculate the stability region for state 
            CA_start.append(CA)
            T_start.append(T)
print("number of initial conditions: {}".format(len(CA_start)))

# convert to np.arrays
CA_start = np.array([CA_start])
T_start = np.array([T_start])
print(CA_start.shape)
x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)  # every row is a pair of initial states within stability region
print("shape of x_deviation is {}".format(x_deviation.shape))
print(x_deviation.shape)  # the initial state is in

number of initial conditions: 56
(1, 56)
shape of x_deviation is (56, 2)
(56, 2)


In [4]:
def CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating CSTR using forward Euler method
    """
    
    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time
    
    C_A = C_A_initial + C_As  # the real state.the derivation plus the steady state
    T = T_initial + T_s
    
    for i in range(int(t_final / t_step)):
        dCAdt = F / V * (C_A0 - C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
        dTdt = F / V * (T_0 - T) - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 + Q / (rho_L * C_p * V)
        
        C_A += dCAdt * t_step
        T += dTdt * t_step
        
        if i%5 ==0:
            C_A_list.append(C_A - C_As)  # in deviation form
            T_list.append(T - T_s)  # in deviation form 
    
    return C_A_list, T_list

In [5]:
# get X and y data for training and testing

CA_output = list()
T_output = list()

CA_input = list()
T_input = list()
CA0_input = list()
Q_input = list()   #input variable for 

for u1 in u1_list:
    C_A0 = u1 + C_A0s
    
    for u2 in u2_list:
        Q = u2 + Q_s
        
        for C_A_initial, T_initial in x_deviation:
            CA0_input.append(u1)
            Q_input.append(u2)
            CA_input.append(C_A_initial)
            T_input.append(T_initial)
            
            C_A_list, T_list = CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial)
            CA_output.append(C_A_list)
            T_output.append(T_list)

In [6]:
# collate input for RNN

CA0_input = np.array(CA0_input)
CA0_input = CA0_input.reshape(-1,1,1)

Q_input = np.array(Q_input)
Q_input = Q_input.reshape(-1,1,1)

CA_input = np.array(CA_input)
CA_input = CA_input.reshape(-1,1,1)

T_input = np.array(T_input)
T_input = T_input.reshape(-1,1,1)

RNN_input = np.concatenate((T_input, CA_input, Q_input, CA0_input), axis=2)   #the value for input variable and the initial value for state variable 

"""
    the input to RNN is in the shape [number of samples x timestep x variables], and the input variables are same for every
    time step, not sure if my treatment here is correct
"""
print("RNN_input shape is {}".format(RNN_input.shape))
RNN_input = RNN_input.repeat(20, axis=1)  # to keep consensus with the shape for RNN_output, since the output variable is collected 100(0.01/1e-4) times for each RNN_input
print("RNN_input shape is {}".format(RNN_input.shape))

RNN_input shape is (50400, 1, 4)
RNN_input shape is (50400, 20, 4)


In [7]:
# collate output for RNN

CA_output = np.array(CA_output)
CA_output = CA_output.reshape(-1, 20, 1)

T_output = np.array(T_output)
T_output = T_output.reshape(-1, 20, 1)

RNN_output = np.concatenate((T_output, CA_output), axis=2)
print("RNN_output shape is {}".format(RNN_output.shape))  # output shape: number of samples x timestep x variables

RNN_output shape is (50400, 20, 2)


In [8]:
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(RNN_input, RNN_output, test_size=0.15, random_state=123)

# define scalers for both X and y base on training data only
scaler_X = preprocessing.StandardScaler().fit(X_train.reshape(-1, 4))
scaler_y = preprocessing.StandardScaler().fit(y_train.reshape(-1, 2))


X_train = scaler_X.transform(X_train.reshape(-1, 4)).reshape(-1,20,4)
X_test = scaler_X.transform(X_test.reshape(-1, 4)).reshape(-1,20,4)
y_train = scaler_y.transform(y_train.reshape(-1,2)).reshape(-1,20,2)
y_test = scaler_y.transform(y_test.reshape(-1,2)).reshape(-1,20,2)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(42840, 20, 4) (7560, 20, 4) (42840, 20, 2) (7560, 20, 2)


In [9]:
model1 = Sequential()
model1.add(SimpleRNN(32, activation='tanh', return_sequences=True))
model1.add(Dense(2, activation='linear'))
model1.compile(optimizer='adam', loss='mse', metrics=['mse'])

t0 = time.time()
history = model1.fit(X_train, y_train, epochs=150, batch_size=256, validation_split=0.25, verbose=2)
t1 = time.time()

Epoch 1/150


2022-10-13 01:37:09.701128: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-13 01:37:09.701170: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-13 01:37:09.701190: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c895b2a47570): /proc/driver/nvidia/version does not exist
2022-10-13 01:37:09.701372: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


126/126 - 2s - loss: 0.1794 - mse: 0.1794 - val_loss: 0.0420 - val_mse: 0.0420 - 2s/epoch - 14ms/step
Epoch 2/150
126/126 - 1s - loss: 0.0242 - mse: 0.0242 - val_loss: 0.0148 - val_mse: 0.0148 - 734ms/epoch - 6ms/step
Epoch 3/150
126/126 - 1s - loss: 0.0107 - mse: 0.0107 - val_loss: 0.0082 - val_mse: 0.0082 - 760ms/epoch - 6ms/step
Epoch 4/150
126/126 - 1s - loss: 0.0066 - mse: 0.0066 - val_loss: 0.0057 - val_mse: 0.0057 - 827ms/epoch - 7ms/step
Epoch 5/150
126/126 - 1s - loss: 0.0050 - mse: 0.0050 - val_loss: 0.0045 - val_mse: 0.0045 - 758ms/epoch - 6ms/step
Epoch 6/150
126/126 - 1s - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0037 - val_mse: 0.0037 - 799ms/epoch - 6ms/step
Epoch 7/150
126/126 - 1s - loss: 0.0034 - mse: 0.0034 - val_loss: 0.0032 - val_mse: 0.0032 - 720ms/epoch - 6ms/step
Epoch 8/150
126/126 - 1s - loss: 0.0029 - mse: 0.0029 - val_loss: 0.0027 - val_mse: 0.0027 - 756ms/epoch - 6ms/step
Epoch 9/150
126/126 - 1s - loss: 0.0025 - mse: 0.0025 - val_loss: 0.0024 - val_mse: 0.

Epoch 65/150
126/126 - 1s - loss: 7.1409e-05 - mse: 7.1409e-05 - val_loss: 8.2612e-05 - val_mse: 8.2612e-05 - 778ms/epoch - 6ms/step
Epoch 66/150
126/126 - 1s - loss: 7.8109e-05 - mse: 7.8109e-05 - val_loss: 7.5224e-05 - val_mse: 7.5224e-05 - 772ms/epoch - 6ms/step
Epoch 67/150
126/126 - 1s - loss: 6.9334e-05 - mse: 6.9334e-05 - val_loss: 7.5524e-05 - val_mse: 7.5524e-05 - 722ms/epoch - 6ms/step
Epoch 68/150
126/126 - 1s - loss: 6.9988e-05 - mse: 6.9988e-05 - val_loss: 6.4663e-05 - val_mse: 6.4663e-05 - 835ms/epoch - 7ms/step
Epoch 69/150
126/126 - 1s - loss: 6.8649e-05 - mse: 6.8649e-05 - val_loss: 6.3535e-05 - val_mse: 6.3535e-05 - 793ms/epoch - 6ms/step
Epoch 70/150
126/126 - 1s - loss: 6.7834e-05 - mse: 6.7834e-05 - val_loss: 6.1165e-05 - val_mse: 6.1165e-05 - 802ms/epoch - 6ms/step
Epoch 71/150
126/126 - 1s - loss: 6.6894e-05 - mse: 6.6894e-05 - val_loss: 5.6543e-05 - val_mse: 5.6543e-05 - 748ms/epoch - 6ms/step
Epoch 72/150
126/126 - 1s - loss: 6.8753e-05 - mse: 6.8753e-05 - val_

Epoch 127/150
126/126 - 1s - loss: 3.1641e-05 - mse: 3.1641e-05 - val_loss: 2.2779e-05 - val_mse: 2.2779e-05 - 835ms/epoch - 7ms/step
Epoch 128/150
126/126 - 1s - loss: 2.3843e-05 - mse: 2.3843e-05 - val_loss: 3.4189e-05 - val_mse: 3.4189e-05 - 750ms/epoch - 6ms/step
Epoch 129/150
126/126 - 1s - loss: 2.4321e-05 - mse: 2.4321e-05 - val_loss: 3.0286e-05 - val_mse: 3.0286e-05 - 832ms/epoch - 7ms/step
Epoch 130/150
126/126 - 1s - loss: 3.0383e-05 - mse: 3.0383e-05 - val_loss: 2.0604e-05 - val_mse: 2.0604e-05 - 734ms/epoch - 6ms/step
Epoch 131/150
126/126 - 1s - loss: 2.7610e-05 - mse: 2.7610e-05 - val_loss: 2.1226e-05 - val_mse: 2.1226e-05 - 802ms/epoch - 6ms/step
Epoch 132/150
126/126 - 1s - loss: 2.5504e-05 - mse: 2.5504e-05 - val_loss: 2.7724e-05 - val_mse: 2.7724e-05 - 749ms/epoch - 6ms/step
Epoch 133/150
126/126 - 1s - loss: 2.9193e-05 - mse: 2.9193e-05 - val_loss: 2.4073e-05 - val_mse: 2.4073e-05 - 748ms/epoch - 6ms/step
Epoch 134/150
126/126 - 1s - loss: 2.5086e-05 - mse: 2.5086e-0

In [10]:
#use the test data to evaluate the model
loss_and_metrics = model1.evaluate(X_test, y_test, batch_size=256)
print(loss_and_metrics)

print(t1-t0)
model1.save('basic.h5')

30/30 [==============================] - 0s 3ms/step - loss: 2.1570e-05 - mse: 2.1570e-05
[2.1569929231191054e-05, 2.1569925593212247e-05]
121.02355551719666
